In [2]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
class CustomEmbeddingModel:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name,device = "cpu")

    @classmethod
    def save_vectors_to_chromadb(cls, embeddings, documents, db_path):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.create_collection("testdb1")
    
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks


with open('ribbon_global.txt', 'r', encoding='utf-8') as file:
    file_contents = file.read()

segmented_text = CustomEmbeddingModel.segment_text(file_contents, chunk_size=50)
# List of models
model_names = [
    'sentence-transformers/all-mpnet-base-v2',
    "BAAI/bge-large-en-v1.5",
    'sentence-transformers/all-MiniLM-L6-v2',
    "thenlper/gte-base",
    "BAAI/bge-base-en-v1.5"  # Add additional model paths if needed
]

# Initialize models and store in a list
models = [CustomEmbeddingModel(model_name=name) for name in model_names]

def generate_batch():
    for i in range(1,6):
        yield i

def embed_text(segmented_text,no,model):
    embeddings = model.encode_text(segmented_text)
    database_path = f"./embedded_v2/path2_{no}"
    model.save_vectors_to_chromadb(embeddings, segmented_text, database_path)
# Encode sentences for each model

import asyncio
count = generate_batch()
task = [embed_text(segmented_text,next(count),model) for model in models]
aysncio.gather(*task)

ChromaDB saved successfully to ./embedded_v2/path2_1


NameError: name 'aysncio' is not defined

In [ ]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer

import requests
from chromadb.utils import embedding_functions
class CustomEmbeddingModel:
    def __init__(self, model_name):
        self.model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

    def encode_text(self, text):
        embeddings = self.model.encode(text, convert_to_tensor=True)
        return embeddings.tolist()

    @classmethod
    def load_chromadb(self,db_path,user_input,embedder1):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection("testdb1",embedding_function=embedder1)
        # return collection  
        results = collection.query(
        query_texts=[user_input],
        n_results=2,
        )
        return results

    def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks
        
model_names = [
    'sentence-transformers/all-mpnet-base-v2',
    "BAAI/bge-large-en-v1.5",
    'sentence-transformers/all-MiniLM-L6-v2',
    "thenlper/gte-base",
    "BAAI/bge-base-en-v1.5"  # Add additional model paths if needed
]
# Initialize models and store in a list
models = [CustomEmbeddingModel(model_name=name) for name in model_names]

user_input = input("User input: ")
context = []
count = 1
for j in models:
    database_path1 = f"./embedded_v2/path_{count}"
    similar_texts = CustomEmbeddingModel.load_chromadb(database_path1,user_input,j.model)
    context.append(similar_texts["documents"][0])
    count += 1

context1 = [' '.join(inner_list) for inner_list in context]
# print(context1)

# # data = set(context)
final_context = " ".join(context1)
print(final_context)
print("##################################################################################")

payload = {
  "message_content": user_input,
  "content": final_context,
  "company_name": "ribbon global",
  "domain": "global services"
}

response = requests.post("http://46.43.144.145:7010/rulebased_bot",json=payload)

print(response.json())